# Retrieval Augemented Generation (RAG)

A Retrievel augmented generation is a form of querying an LLM by providing some documents for additional context. For example, if you want the LLM to answer some questions based on specific documents you have, you can write a RAG model to do just that. 

A RAG model is primarily made up of two components, the retriever, and the reader

In [ ]:
from langchain.docstore.document import Document
from langchain_dartmouth.retrievers.document_compressors import DartmouthReranker
import os

The directory ./rag_documents contains different documents on various topics. Using the `TextLoader` from langchain, we can import these documents. Langchain has various different loaders for different file types. You can learn more about them [here](https://python.langchain.com/docs/integrations/document_loaders/)

In [ ]:
directory_to_search = "./rag_documents"

document_directories = [file for file in os.listdir(directory_to_search) if file.endswith('.txt')]
document_directories = [os.path.join(directory_to_search, file) for file in document_directories]
print(document_directories)


In [ ]:
from langchain.document_loaders import TextLoader

docs = []
for directory in document_directories:
    text_loader = TextLoader(directory, encoding='utf-8')
    docs += text_loader.load()



Given a **collection** of documents, we are interested in finding the ones most relevent to our question. To determine this, we can use the `DartmouthReranker` function.


In [ ]:
reranker = DartmouthReranker()

In [ ]:
from langchain_dartmouth.llms import ChatDartmouth

query = "Are asteroids going to hit me?"
ranked_docs = reranker.compress_documents(query=query, documents=docs)

for doc in ranked_docs:
    print(doc.metadata['source'])

<div class="alert alert-info">

**Note:** We can see that when our query is related to asteroids, the document correcly ranks the asteroids.txt as the most relevent document. Now, we can prepend the content of the first ranked document to our query to get a response which considers the information in the document
</div>

In [ ]:
llm = ChatDartmouth(model_name="llama-3-1-8b-instruct")
response = llm.invoke(ranked_docs[0].page_content + 'Considering this, answer the following question:' +  query)
print(response)

# Summary
In this recipe we use a query and `DartmouthReranker` to retrieve relative documents from a **collection**. We then content of the highest ranked document is prepended in a prompt to the llm. This is an implementation of retreival augmented generation.